## Model Penentu Barang

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from datetime import datetime

In [2]:
# Data
df_produk = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/produk_v4.csv')
df_toko = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/toko.csv')
df_transaksi = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/transaksi_v4.csv')


In [3]:
# 2.1. Konversi Tipe Data & Tentukan Tanggal Referensi
# Untuk konsistensi, kita anggap 'hari ini' adalah satu hari setelah transaksi terakhir di dataset
df_produk['expire_date'] = pd.to_datetime(df_produk['expire_date'])
df_transaksi['tanggal_transaksi'] = pd.to_datetime(df_transaksi['tanggal_transaksi'])
HARI_INI = df_transaksi['tanggal_transaksi'].max() + pd.Timedelta(days=1)
print(f"Tanggal referensi (HARI_INI) ditetapkan pada: {HARI_INI.date()}")

# 2.2. Membuat Fitur Agregat dari Tabel Transaksi
# Kita hitung metrik penjualan untuk setiap produk
print("\nMenghitung fitur agregat dari transaksi...")
agg_transaksi = df_transaksi.groupby('id_produk').agg(
    total_penjualan=('id_produk', 'count'),
    penjualan_terakhir=('tanggal_transaksi', 'max'),
    jumlah_hari_jual=('tanggal_transaksi', 'nunique')
).reset_index()

# Hitung penjualan harian rata-rata
# Tambah 1 untuk menghindari pembagian dengan nol
agg_transaksi['penjualan_harian_avg'] = agg_transaksi['total_penjualan'] / (agg_transaksi['jumlah_hari_jual'] + 1)
agg_transaksi['hari_sejak_penjualan_terakhir'] = (HARI_INI - agg_transaksi['penjualan_terakhir']).dt.days

print("Fitur agregat berhasil dibuat.")
display(agg_transaksi.head())


# 2.3. Membuat Fitur dari Tabel Produk & Menggabungkan Data
print("\nMenggabungkan semua data menjadi satu DataFrame...")
# Mulai dengan df_produk sebagai basis
df_model = df_produk.copy()

# Gabungkan dengan fitur agregat transaksi
df_model = pd.merge(df_model, agg_transaksi[['id_produk', 'penjualan_harian_avg', 'hari_sejak_penjualan_terakhir']], on='id_produk', how='left')

# Isi nilai NaN untuk produk yang mungkin belum pernah terjual
df_model['penjualan_harian_avg'].fillna(0, inplace=True)
# Jika belum pernah terjual, kita anggap sudah sangat lama
df_model['hari_sejak_penjualan_terakhir'].fillna(999, inplace=True)


# Buat fitur utama yang akan kita gunakan
df_model['hari_menuju_kedaluwarsa'] = (df_model['expire_date'] - HARI_INI).dt.days
# Fitur 'margin_headroom' mengukur seberapa banyak ruang yang kita miliki untuk diskon
# Kita asumsikan minimal_margin ada di data produk atau bisa di-join dari tabel kategori
# Untuk contoh ini, kita buat nilai dummy
df_model['minimal_margin'] = df_model['margin'] * 0.4 # Contoh: minimal margin adalah 40% dari margin saat ini
df_model['margin_headroom'] = df_model['margin'] - df_model['minimal_margin']

# Pilih fitur-fitur numerik yang relevan untuk model
fitur_model = [
    'margin',
    'hari_jual_minimal',
    'penjualan_harian_avg',
    'hari_sejak_penjualan_terakhir',
    'hari_menuju_kedaluwarsa',
    'margin_headroom'
]

# Tampilkan hasil akhir dari data yang siap dimodelkan
print("\nData akhir yang siap untuk pemodelan:")
display(df_model[['id_produk', 'nama_produk'] + fitur_model].head())

Tanggal referensi (HARI_INI) ditetapkan pada: 2025-03-01

Menghitung fitur agregat dari transaksi...
Fitur agregat berhasil dibuat.


,id_produk,total_penjualan,penjualan_terakhir,jumlah_hari_jual,penjualan_harian_avg,hari_sejak_penjualan_terakhir
0,P00000001,19,2024-10-25,19,0.950000,127
1,P00000002,20,2025-01-29,20,0.952381,31
2,P00000003,14,2025-02-25,14,0.933333,4
3,P00000004,19,2025-02-06,19,0.950000,23
4,P00000005,26,2025-01-13,26,0.962963,47



Menggabungkan semua data menjadi satu DataFrame...

Data akhir yang siap untuk pemodelan:


/tmp/ipython-input-3387686918.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['penjualan_harian_avg'].fillna(0, inplace=True)
/tmp/ipython-input-3387686918.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

,id_produk,nama_produk,margin,hari_jual_minimal,penjualan_harian_avg,hari_sejak_penjualan_terakhir,hari_menuju_kedaluwarsa,margin_headroom
0,P00000001,JayaMakmur Garam Varian C 50g,0.1845,19,0.950000,127.0,552,0.11070
1,P00000002,IndoAgro Sarden Kaleng Spesial 500g,0.1935,29,0.952381,31.0,469,0.11610
2,P00000003,SentosaNusantara Kopi Kemasan Varian A 750ml,0.2955,25,0.933333,4.0,562,0.17730
3,P00000004,NusantaraPT Makaroni Spesial 200g,0.2411,26,0.950000,23.0,439,0.14466
4,P00000005,IndoJaya Yogurt Spesial 750ml,0.2222,30,0.962963,47.0,408,0.13332


In [4]:
# ==============================================================================
# LANGKAH 3 (REVISI BESAR): Membuat Label Skor Urgensi Kontinu
# ==============================================================================
print("Membuat label 'urgency_score' yang lebih bernuansa...")

# Tentukan bobot bisnis (bisa disesuaikan)
W_KEDALUWARSA = 0.6  # Paling penting
W_KELAMBATAN = 0.3  # Cukup penting
W_PENJUALAN = 0.1   # Sebagai penalti

# Hindari pembagian dengan nol atau nilai negatif untuk hari menuju kedaluwarsa
# Kita batasi nilai minimumnya menjadi 1
df_model['hari_menuju_kedaluwarsa_safe'] = df_model['hari_menuju_kedaluwarsa'].clip(lower=1)

# Hitung komponen skor
# Skor kedaluwarsa (semakin kecil harinya, semakin besar skornya, non-linear)
df_model['skor_kedaluwarsa'] = 1 / df_model['hari_menuju_kedaluwarsa_safe']

# Skor kelambatan (semakin lama tidak laku, semakin tinggi skornya)
df_model['skor_kelambatan'] = df_model['hari_sejak_penjualan_terakhir']

# Skor penalti penjualan (semakin laku, semakin tinggi penaltinya)
df_model['skor_penalti_penjualan'] = df_model['penjualan_harian_avg']

# Gabungkan menjadi skor urgensi mentah
df_model['urgency_score_raw'] = (W_KEDALUWARSA * df_model['skor_kedaluwarsa'] +
                                 W_KELAMBATAN * df_model['skor_kelambatan'] -
                                 W_PENJUALAN * df_model['skor_penalti_penjualan'])

# Normalisasi skor mentah ke rentang 0-100 agar lebih mudah diinterpretasikan
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 100))
df_model['urgency_score'] = scaler.fit_transform(df_model[['urgency_score_raw']])

print("Distribusi 'urgency_score' yang baru:")
print(df_model['urgency_score'].describe())


# ==============================================================================
# LANGKAH 4 & 5 (REVISI BESAR): Melatih Model Regresi untuk Memprediksi Skor
# ==============================================================================
# Karena target kita sekarang kontinu, kita gunakan LGBMRegressor.
# Tujuannya adalah melatih model yang bisa memprediksi 'urgency_score' untuk produk baru
# atau saat data diperbarui.

# Definisikan fitur (X) dan target (y) BARU kita
X = df_model[fitur_model]  # Fitur tetap sama
y = df_model['urgency_score']   # Target sekarang adalah skor kontinu

# Bagi data menjadi set training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi {len(X_train)} baris training dan {len(X_test)} baris testing.")

# Inisialisasi model REGRESI
regressor = lgb.LGBMRegressor(
    objective='regression_l1',  # L1 (MAE) loss, lebih tahan terhadap outlier
    metric='mae',
    n_estimators=1000,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
    num_leaves=31
)

print("\nMemulai pelatihan model LGBMRegressor...")
regressor.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(20, verbose=False)]
)
print("Pelatihan model regresi selesai.")

# ==============================================================================
# LANGKAH 6 (REVISI BESAR): Membuat Peringkat Berdasarkan Prediksi Skor
# ==============================================================================
print("\nMemprediksi skor urgensi untuk semua produk...")

# Gunakan model regresi untuk memprediksi skor pada keseluruhan dataset
df_model['skor_prediksi'] = regressor.predict(X)

# Buat DataFrame hasil
df_hasil_peringkat_baru = df_model[[
    'id_produk', 'nama_produk', 'kategori_produk', 'skor_prediksi',
    'hari_menuju_kedaluwarsa', 'hari_sejak_penjualan_terakhir'
]].copy()

# Urutkan berdasarkan SKOR PREDIKSI dari yang tertinggi ke terendah
df_hasil_peringkat_baru = df_hasil_peringkat_baru.sort_values(by='skor_prediksi', ascending=False)

print("\n\n--- HASIL AKHIR (VERSI BARU): TOP 20 PRODUK KANDIDAT UNTUK DISKON ---")
display(df_hasil_peringkat_baru.head(20))

print("\nContoh sebaran skor prediksi yang unik:")
print(np.unique(df_hasil_peringkat_baru['skor_prediksi'].round(2)))

Membuat label 'urgency_score' yang lebih bernuansa...
Distribusi 'urgency_score' yang baru:
count    87506.000000
mean        30.025333
std         39.210563
min          0.000000
25%          3.312699
50%          8.830344
75%         35.278378
max        100.000000
Name: urgency_score, dtype: float64

Data dibagi menjadi 70004 baris training dan 17502 baris testing.

Memulai pelatihan model LGBMRegressor...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1110
[LightGBM] [Info] Number of data points in the train set: 70004, number of used features: 6
[LightGBM] [Info] Start training from score 8.824883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

,id_produk,nama_produk,kategori_produk,skor_prediksi,hari_menuju_kedaluwarsa,hari_sejak_penjualan_terakhir
87492,P00087493,PTAgro Buah-Buahan Varian C 200g,Buah-Buahan,99.999852,178,999.0
55494,P00055495,AbadiAbadi Seafood Segar Varian B 750g,Seafood Segar,99.999852,182,999.0
74435,P00074436,AbadiMakmur Buah-Buahan Varian A 200g,Buah-Buahan,99.999852,178,999.0
83370,P00083371,AgroPT Seafood Segar Varian C 200g,Seafood Segar,99.999852,188,999.0
74458,P00074459,CVNusantara Buah-Buahan Varian C 250g,Buah-Buahan,99.999852,181,999.0
74470,P00074471,SentosaCV Daging Segar Varian C 200g,Daging Segar,99.999852,191,999.0
59369,P00059370,PTJaya Sayur-Sayuran Spesial 500g,Sayur-Sayuran,99.999852,179,999.0
59361,P00059362,AbadiAbadi Roti Varian B 100g,Roti,99.999852,194,999.0
74512,P00074513,CVJaya Seafood Segar Spesial 750g,Seafood Segar,99.999852,180,999.0
55572,P00055573,SentosaMakmur Buah-Buahan Spesial 100g,Buah-Buahan,99.999852,193,999.0



Contoh sebaran skor prediksi yang unik:
[1.000e-02 2.000e-02 1.000e-01 ... 5.807e+01 5.810e+01 1.000e+02]


In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np # Import numpy

# Asumsikan df_hasil_peringkat_baru sudah berisi skor prediksi

# --- LANGKAH 1: Tentukan Parameter ---
TOTAL_SLOT_PROMOSI = 1000
SKOR_THRESHOLD = 50 # Angka ini bisa disesuaikan

# --- LANGKAH 2: Hitung Kandidat per Kategori ---
# Tandai produk mana yang dianggap 'kandidat kuat'
df_hasil_peringkat_baru['is_candidate'] = df_hasil_peringkat_baru['skor_prediksi'] > SKOR_THRESHOLD

# Hitung jumlah kandidat per kategori
candidate_counts = df_hasil_peringkat_baru[df_hasil_peringkat_baru['is_candidate']].groupby('kategori_produk').size()
total_candidates = candidate_counts.sum()

print("Jumlah kandidat kuat (skor > 50) per kategori:")
print(candidate_counts)

# --- LANGKAH 3: Hitung dan Alokasikan Kuota Dinamis ---
# Hitung kuota mentah berdasarkan proporsi
raw_quotas = (candidate_counts / total_candidates) * TOTAL_SLOT_PROMOSI

# Bulatkan kuota dan pastikan jumlahnya tepat 30 (logika pembulatan sederhana)
# Kita bulatkan ke bawah, lalu sisa slotnya kita berikan satu per satu ke kategori dengan desimal terbesar
quotas = np.floor(raw_quotas).astype(int) # Menggunakan np.floor()
sisa_slot = TOTAL_SLOT_PROMOSI - quotas.sum()

# Berikan sisa slot ke kategori dengan sisa desimal terbesar
sisa_desimal = raw_quotas - quotas
# Sort sisa desimal dari terbesar ke terkecil dan ambil indexnya
top_sisa_cats = sisa_desimal.sort_values(ascending=False).index.tolist()

# Alokasikan sisa slot
for cat in top_sisa_cats:
    if sisa_slot <= 0:
        break
    quotas[cat] += 1
    sisa_slot -= 1

# Pastikan setiap kategori yang punya kandidat dapat minimal 1 slot (opsional, tergantung aturan bisnis)
# Jika ada kategori dengan kandidat tapi kuota 0, alokasikan 1 (ambil dari kuota terbesar jika perlu)
# Untuk saat ini, kita biarkan bisa 0 jika proporsinya sangat kecil

print("\nKuota Dinamis yang Dialokasikan:")
print(quotas)

# --- LANGKAH 4: Ambil Produk Berdasarkan Kuota Dinamis ---
list_of_dfs = []
for category, num_to_take in quotas.items():
    if num_to_take > 0: # Hanya ambil jika kuotanya lebih dari 0
        # Ambil 'num_to_take' produk teratas dari kategori tersebut
        top_products_per_cat = (
            df_hasil_peringkat_baru[df_hasil_peringkat_baru['kategori_produk'] == category]
            .sort_values(by='skor_prediksi', ascending=False) # Pastikan diurutkan berdasarkan skor prediksi
            .head(num_to_take)
        )
        list_of_dfs.append(top_products_per_cat)

# Gabungkan semua hasilnya menjadi satu DataFrame
if list_of_dfs: # Cek apakah list tidak kosong sebelum concat
    df_final_dinamis = pd.concat(list_of_dfs)
else:
    df_final_dinamis = pd.DataFrame() # Buat DataFrame kosong jika tidak ada produk yang dipilih


# Urutkan hasil akhir berdasarkan skor
df_final_dinamis = df_final_dinamis.sort_values(by='skor_prediksi', ascending=False)

print(f"\n--- DAFTAR FINAL {len(df_final_dinamis)} PRODUK DENGAN KUOTA DINAMIS ---")
display(df_final_dinamis)

Jumlah kandidat kuat (skor > 50) per kategori:
kategori_produk
Air Mineral         539
Beras               441
Bihun               415
Biskuit             292
Buah Kering         432
Buah-Buahan         463
Cokelat             323
Daging Segar        453
Garam               412
Gula                468
Gulali              346
Ice Cream           446
Jus Kemasan         409
Kacang              417
Kaldu Jamur         355
Kecap               397
Keju                445
Kentang Goreng      443
Keripik             414
Kopi Bubuk          447
Kopi Kemasan        435
Kornet              421
Krim                324
Kuaci               345
Makaroni            470
Marshmallow         434
Mayones             434
Mentega             375
Mie Instan          445
Minuman Isotonik    428
Minyak Goreng       431
Nugget              474
Pasta               429
Penyedap Rasa       373
Permen              480
Roti                326
Saos                498
Sarden Kaleng       363
Sayur-Sayuran       482
S

,id_produk,nama_produk,kategori_produk,skor_prediksi,hari_menuju_kedaluwarsa,hari_sejak_penjualan_terakhir,is_candidate
59361,P00059362,AbadiAbadi Roti Varian B 100g,Roti,99.999852,194,999.0,True
80128,P00080129,MakmurIndo Roti Spesial 100g,Roti,99.999852,175,999.0,True
66236,P00066237,NusantaraPangan Roti Ekstra 50g,Roti,99.999852,184,999.0,True
66232,P00066233,IndoNusantara Roti Varian C 100g,Roti,99.999852,177,999.0,True
65774,P00065775,CVPangan Roti Varian C 500g,Roti,99.999852,188,999.0,True
...,...,...,...,...,...,...,...
77673,P00077674,CVAbadi Soda Varian C 1500ml,Soda,99.999359,374,999.0,True
86628,P00086629,PanganMakmur Mie Instan Spesial 1000g,Mie Instan,99.999359,374,999.0,True
75083,P00075084,AbadiSentosa Mie Instan Varian A 500g,Mie Instan,99.999359,374,999.0,True
86343,P00086344,PanganAgro Mie Instan Spesial 100g,Mie Instan,99.999359,375,999.0,True


## Model Penentu Diskon

In [6]:
# Muat semua data yang dibutuhkan
df_transaksi = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/transaksi_v4.csv')
df_produk = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/produk_v4.csv')
df_toko = pd.read_csv('https://raw.githubusercontent.com/AzrilFahmiardi/bizzit-compfest-2025/refs/heads/master/data/toko.csv')
df_top_30_rekomendasi = df_final_dinamis.copy()

In [7]:
import pandas as pd
import numpy as np

print("--- Langkah 1: Membuat df_master_train (Menggabungkan data historis) ---")

# Menyeragamkan tipe data kunci untuk memastikan merge berhasil
df_transaksi['id_produk'] = df_transaksi['id_produk'].astype(str)
df_produk['id_produk'] = df_produk['id_produk'].astype(str)
df_toko['id_toko'] = df_toko['id_toko'].astype(int)
df_transaksi['id_toko'] = df_transaksi['id_toko'].astype(int)

# Gabungkan transaksi dengan data produk (menghapus kolom duplikat `id_toko` dari produk)
print("   - Menggabungkan transaksi dengan produk...")
if 'id_toko' in df_produk.columns:
    df_produk_cleaned = df_produk.drop(columns=['id_toko'])
else:
    df_produk_cleaned = df_produk
df_master_train = pd.merge(df_transaksi, df_produk_cleaned, on='id_produk', how='left')

# Gabungkan dengan data toko
print("   - Menggabungkan dengan data toko...")
df_master_train = pd.merge(df_master_train, df_toko, on='id_toko', how='left')

print("\n'df_master_train' (buku pelajaran untuk model) berhasil dibuat!")
print("Shape:", df_master_train.shape)
display(df_master_train.head())

--- Langkah 1: Membuat df_master_train (Menggabungkan data historis) ---
   - Menggabungkan transaksi dengan produk...
   - Menggabungkan dengan data toko...

'df_master_train' (buku pelajaran untuk model) berhasil dibuat!
Shape: (607561, 38)


,id_produk,minggu,tanggal_transaksi,current_event,id_toko,harga_promosi,diskon,tipe_diskon,margin_promosi,hari_jual,...,jumlah_karyawan,tipe,umur_konsumen,jenis_kelamin_konsumen,penghasilan_konsumen_avg,pekerjaan_konsumen,kebiasaan_konsumen,reaksi_promo,frekuensi_pembelian,waktu_pembelian
0,P00006032,1,2023-01-01,Promo Akhir Pekan,1,17100,0.1115,Generic Product Discount,0.0382,171,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
1,P00001069,1,2023-01-01,Promo Akhir Pekan,1,5743,0.1592,Generic Product Discount,0.0350,118,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
2,P00001268,1,2023-01-01,Promo Akhir Pekan,1,16400,0.0744,Generic Product Discount,0.1013,78,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
3,P00005220,1,2023-01-01,Promo Akhir Pekan,1,12800,0.1852,Generic Product Discount,0.0353,15,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
4,P00004992,1,2023-01-01,Promo Akhir Pekan,1,16700,0.1132,Generic Product Discount,0.0354,100,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"


In [8]:
print("\n--- Langkah 2: Mendefinisikan 'outcome' (profit_transaksi) ---")

df_master_train['profit_transaksi'] = df_master_train['harga_promosi'] - df_master_train['harga_beli']

print("Kolom 'profit_transaksi' berhasil dibuat di df_master_train.")


--- Langkah 2: Mendefinisikan 'outcome' (profit_transaksi) ---
Kolom 'profit_transaksi' berhasil dibuat di df_master_train.


In [9]:
print("\n--- Langkah 3: Feature Engineering pada Data Latihan ---")

# 3a. Parsing Fitur Konteks Toko
print("   - Mengekstrak fitur dari kolom Toko...")
try:
    df_master_train['rasio_pekerja_kantoran'] = df_master_train['pekerjaan_konsumen'].str.extract(r'pekerja_kantoran: (\d+\.\d+)').astype(float).fillna(0)
    df_master_train['rasio_impulsif'] = df_master_train['kebiasaan_konsumen'].str.extract(r'impulsif: (\d+\.\d+)').astype(float).fillna(0)
except Exception as e:
    print(f"   - Peringatan: Gagal mengekstrak fitur toko. Error: {e}")

# 3b. Membuat Fitur Baru dari Detail Produk
print("   - Membuat fitur 'gramasi' dan 'harga_per_gram'...")
df_master_train['gramasi'] = df_master_train['nama_produk'].str.extract(r'(\d+)\s?(g|ml)')[0].astype(float).fillna(0)
df_master_train['harga_per_gram'] = (df_master_train['harga_jual'] / df_master_train['gramasi']).replace([np.inf, -np.inf], 0).fillna(0)

# 3c. One-Hot Encoding Fitur Kategorikal
print("   - Melakukan One-Hot Encoding...")
df_master_train_featured = pd.get_dummies(df_master_train, columns=['brand', 'kategori_produk', 'current_event'], prefix=['brand', 'kat', 'event'])

print("Feature engineering pada data latihan selesai.")


--- Langkah 3: Feature Engineering pada Data Latihan ---
   - Mengekstrak fitur dari kolom Toko...
   - Membuat fitur 'gramasi' dan 'harga_per_gram'...
   - Melakukan One-Hot Encoding...
Feature engineering pada data latihan selesai.


In [10]:
print("\n--- Langkah 4: Finalisasi dan Pemisahan Data Latihan ---")

# Definisikan semua nama kolom yang akan menjadi fitur (X)
fitur_numerik = [
    'harga_jual', 'margin', 'hari_jual', 'kedaluwarsa',
    'rasio_pekerja_kantoran', 'rasio_impulsif', 'gramasi', 'harga_per_gram'
]
fitur_ohe = [col for col in df_master_train_featured.columns if col.startswith(('brand_', 'kat_', 'event_'))]
feature_columns = fitur_numerik + fitur_ohe
print(f"   - Total {len(feature_columns)} fitur akan digunakan.")

# Pisahkan data menjadi beberapa bagian berdasarkan 'tipe_diskon'
dataframes_per_treatment = {}
for treatment_name in df_master_train_featured['tipe_diskon'].unique():
    dataframes_per_treatment[treatment_name] = df_master_train_featured[df_master_train_featured['tipe_diskon'] == treatment_name].copy()

print("\nData latihan telah dipisahkan per tipe diskon:")
for name, df in dataframes_per_treatment.items():
    print(f"- {name}: {len(df)} baris")


--- Langkah 4: Finalisasi dan Pemisahan Data Latihan ---
   - Total 162 fitur akan digunakan.

Data latihan telah dipisahkan per tipe diskon:
- Generic Product Discount: 198079 baris
- Expired Discount: 69952 baris
- BOGO: 887 baris
- Tanpa Diskon: 247805 baris
- Discount Perkenalan: 4889 baris
- Event Based Discount: 85949 baris


In [11]:
import lightgbm as lgb
print("\n--- Langkah 5: Melatih Model-model T-Learner ---")
trained_models = {}
for treatment_name, df_treatment in dataframes_per_treatment.items():
    print(f"--> Melatih model untuk: '{treatment_name}'")

    if len(df_treatment) < 100:
        print(f"    - Peringatan: Data terlalu sedikit, model untuk '{treatment_name}' dilewati.")
        continue

    X_train = df_treatment[feature_columns]
    y_train = df_treatment['profit_transaksi']

    model = lgb.LGBMRegressor(random_state=42, n_estimators=100, verbose=-1)
    model.fit(X_train, y_train)

    trained_models[treatment_name] = model
    print(f"    - Model untuk '{treatment_name}' berhasil dilatih.")

print("\n=================================================")
print("  PROSES PELATIHAN MODEL SPESIALIS SELESAI  ")
print("=================================================")


--- Langkah 5: Melatih Model-model T-Learner ---
--> Melatih model untuk: 'Generic Product Discount'
    - Model untuk 'Generic Product Discount' berhasil dilatih.
--> Melatih model untuk: 'Expired Discount'
    - Model untuk 'Expired Discount' berhasil dilatih.
--> Melatih model untuk: 'BOGO'
    - Model untuk 'BOGO' berhasil dilatih.
--> Melatih model untuk: 'Tanpa Diskon'
    - Model untuk 'Tanpa Diskon' berhasil dilatih.
--> Melatih model untuk: 'Discount Perkenalan'
    - Model untuk 'Discount Perkenalan' berhasil dilatih.
--> Melatih model untuk: 'Event Based Discount'
    - Model untuk 'Event Based Discount' berhasil dilatih.

  PROSES PELATIHAN MODEL SPESIALIS SELESAI  


In [12]:
print("--- Langkah 6: Menyiapkan Data untuk Diberi Rekomendasi (Versi Perbaikan) ---")

# Ambil daftar ID produk dari hasil Model 1
list_id_rekomendasi = df_top_30_rekomendasi['id_produk'].astype(str).unique().tolist()

# Mulai dengan DataFrame yang berisi produk-produk ini beserta detailnya dari df_produk
df_rekomendasi = df_produk[df_produk['id_produk'].isin(list_id_rekomendasi)].copy()

# Gabungkan dengan data toko
df_rekomendasi = pd.merge(df_rekomendasi, df_toko, on='id_toko', how='left')

print(f"   - Menyiapkan data untuk {len(df_rekomendasi)} kombinasi produk-toko...")

# ============================ SOLUSI DI SINI ============================
# [SOLUSI] Tambahkan kolom 'current_event' secara manual untuk simulasi.
# Kita asumsikan promosi akan dijalankan pada konteks 'Promo Akhir Pekan'.
df_rekomendasi['current_event'] = 'Promo Akhir Pekan'
print("   - Kolom 'current_event' ditambahkan dengan nilai 'Promo Akhir Pekan'.")
# =======================================================================


# Lakukan feature engineering yang SAMA PERSIS dengan Langkah 3
print("   - Menerapkan feature engineering...")

# 6a. Parsing Fitur Konteks Toko
try:
    df_rekomendasi['rasio_pekerja_kantoran'] = df_rekomendasi['pekerjaan_konsumen'].str.extract(r'pekerja_kantoran: (\d+\.\d+)').astype(float).fillna(0)
    df_rekomendasi['rasio_impulsif'] = df_rekomendasi['kebiasaan_konsumen'].str.extract(r'impulsif: (\d+\.\d+)').astype(float).fillna(0)
except Exception: pass

# 6b. Membuat Fitur Baru dari Detail Produk
df_rekomendasi['gramasi'] = df_rekomendasi['nama_produk'].str.extract(r'(\d+)\s?(g|ml)')[0].astype(float).fillna(0)
df_rekomendasi['harga_per_gram'] = (df_rekomendasi['harga_jual'] / df_rekomendasi['gramasi']).replace([np.inf, -np.inf], 0).fillna(0)
df_rekomendasi['hari_jual'] = df_rekomendasi['hari_jual_minimal']
df_rekomendasi['kedaluwarsa'] = 365 - df_rekomendasi['hari_jual_minimal']


# 6c. One-Hot Encoding (Sekarang akan berhasil)
df_rekomendasi_featured = pd.get_dummies(df_rekomendasi, columns=['brand', 'kategori_produk', 'current_event'], prefix=['brand', 'kat', 'event'])

# 6d. Finalisasi - Pastikan semua kolom ada dan urutannya sama
for col in feature_columns:
    if col not in df_rekomendasi_featured.columns:
        df_rekomendasi_featured[col] = 0
X_rekomendasi = df_rekomendasi_featured[feature_columns]

print("Data rekomendasi siap untuk diprediksi.")

--- Langkah 6: Menyiapkan Data untuk Diberi Rekomendasi (Versi Perbaikan) ---
   - Menyiapkan data untuk 1000 kombinasi produk-toko...
   - Kolom 'current_event' ditambahkan dengan nilai 'Promo Akhir Pekan'.
   - Menerapkan feature engineering...
Data rekomendasi siap untuk diprediksi.


In [13]:
print("\n--- Langkah 7: Simulasi, Perhitungan Uplift, dan Rekomendasi Final (Versi Perbaikan) ---")

# ==============================================================================
# Tahap 1 & 2 (Tidak berubah)
# ==============================================================================
print("   - Tahap 1 & 2: Menyiapkan data dan melakukan prediksi...")
# (Kode dari langkah 6 untuk menyiapkan X_rekomendasi diasumsikan sudah berjalan)

hasil_prediksi = {}
for treatment_name, model in trained_models.items():
    prediksi = model.predict(X_rekomendasi)
    hasil_prediksi[treatment_name] = prediksi

df_prediksi_profit = pd.DataFrame(hasil_prediksi, index=df_rekomendasi.index)


# ==============================================================================
# Tahap 3: Hitung Uplift dan Tentukan Strategi Terbaik (Logika Diperbaiki)
# ==============================================================================
print("   - Tahap 3: Menghitung uplift dan memilih strategi terbaik...")

if 'Tanpa Diskon' in df_prediksi_profit.columns:
    baseline_profit = df_prediksi_profit['Tanpa Diskon']
    # df_uplift_numeric hanya berisi kolom-kolom uplift yang berupa angka
    df_uplift_numeric = df_prediksi_profit.drop(columns=['Tanpa Diskon'], errors='ignore').subtract(baseline_profit, axis=0)
else:
    print("   - Peringatan: Model 'Tanpa Diskon' tidak ditemukan.")
    df_uplift_numeric = df_prediksi_profit
    baseline_profit = 0

# [DIUBAH] Buat DataFrame hasil untuk menampung rekomendasi
df_hasil_rekomendasi = pd.DataFrame(index=df_uplift_numeric.index)

# [DIUBAH] 1. Cari nama strategi terbaik DULU (dari DataFrame angka)
df_hasil_rekomendasi['rekomendasi_strategi'] = df_uplift_numeric.idxmax(axis=1)

# [DIUBAH] 2. BARU ambil nilai uplift tertinggi
df_hasil_rekomendasi['estimasi_uplift_profit'] = df_uplift_numeric.max(axis=1)

# 3. Logika untuk menangani uplift negatif (tidak berubah)
df_hasil_rekomendasi.loc[df_hasil_rekomendasi['estimasi_uplift_profit'] < 0, 'rekomendasi_strategi'] = 'Tanpa Diskon'
df_hasil_rekomendasi.loc[df_hasil_rekomendasi['estimasi_uplift_profit'] < 0, 'estimasi_uplift_profit'] = 0


# ==============================================================================
# Tahap 4: Tampilkan Hasil Akhir (Tidak berubah)
# ==============================================================================
print("\n============================================================")
print("         HASIL AKHIR: REKOMENDASI STRATEGI DISKON           ")
print("============================================================")

df_final_recommendation = pd.concat([
    df_rekomendasi[['id_produk', 'nama_produk', 'kategori_produk', 'id_toko']].reset_index(drop=True),
    df_hasil_rekomendasi[['rekomendasi_strategi', 'estimasi_uplift_profit']].reset_index(drop=True)
], axis=1)

df_agg_recommendation = df_final_recommendation.groupby(['id_produk', 'nama_produk', 'kategori_produk']).agg(
    rekomendasi_utama=('rekomendasi_strategi', lambda x: x.mode()[0] if not x.empty else "N/A"),
    rata_rata_uplift_profit=('estimasi_uplift_profit', 'mean')
).reset_index().sort_values(by='rata_rata_uplift_profit', ascending=False)


display(df_agg_recommendation.head(30).style.format({'rata_rata_uplift_profit': 'Rp {:,.0f}'.format}).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))


--- Langkah 7: Simulasi, Perhitungan Uplift, dan Rekomendasi Final (Versi Perbaikan) ---
   - Tahap 1 & 2: Menyiapkan data dan melakukan prediksi...
   - Tahap 3: Menghitung uplift dan memilih strategi terbaik...

         HASIL AKHIR: REKOMENDASI STRATEGI DISKON           


,id_produk,nama_produk,kategori_produk,rekomendasi_utama,rata_rata_uplift_profit
982,P00087170,AbadiCV Daging Segar Spesial 50g,Daging Segar,Event Based Discount,"Rp 1,311"
888,P00084404,JayaPangan Beras Varian A 500g,Beras,Event Based Discount,Rp 825
4,P00054344,PTJaya Daging Segar Spesial 1000g,Daging Segar,Event Based Discount,Rp 761
18,P00055495,AbadiAbadi Seafood Segar Varian B 750g,Seafood Segar,Event Based Discount,Rp 730
842,P00083371,AgroPT Seafood Segar Varian C 200g,Seafood Segar,Event Based Discount,Rp 603
467,P00074514,NusantaraPT Seafood Segar Ekstra 200g,Seafood Segar,Event Based Discount,Rp 595
160,P00065969,JayaPangan Seafood Segar Varian C 50g,Seafood Segar,Event Based Discount,Rp 542
323,P00071462,PanganIndo Buah Kering Spesial 100g,Buah Kering,Event Based Discount,Rp 537
143,P00065063,SentosaIndo Buah Kering Varian B 100g,Buah Kering,Event Based Discount,Rp 457
700,P00079809,AgroCV Daging Segar Ekstra 750g,Daging Segar,Event Based Discount,Rp 450


In [14]:
# import pandas as pd
# import numpy as np
# from datetime import datetime, timedelta

# print("\n--- MEMULAI PROSES PENGAYAAN REKOMENDASI (FINAL & SELARAS) ---")

# # ==============================================================================
# # LANGKAH 1: PERSIAPAN DATA & KONTEKS WAKTU
# # ==============================================================================
# # Gunakan df_agg_recommendation dari output T-Learner Anda sebagai dasar
# df_final = df_agg_recommendation.copy()
# print(f"   - Memulai dengan {len(df_final)} produk dari hasil T-Learner.")

# # Gabungkan dengan df_produk untuk mendapatkan fitur yang dibutuhkan
# df_final['id_produk'] = df_final['id_produk'].astype(str)
# df_produk['id_produk'] = df_produk['id_produk'].astype(str)
# fitur_tambahan = ['id_produk', 'harga_jual', 'harga_kompetitor', 'produk_musiman', 'hari_jual_minimal', 'expire_date']
# df_final = pd.merge(df_final, df_produk[fitur_tambahan], on='id_produk', how='left')

# # Siapkan konteks waktu
# HARI_INI = pd.to_datetime(df_transaksi['tanggal_transaksi']).max() + pd.Timedelta(days=1)
# df_final['expire_date'] = pd.to_datetime(df_final['expire_date'])
# df_final['hari_menuju_kedaluwarsa'] = (df_final['expire_date'] - HARI_INI).dt.days
# events_calendar = {"Ramadan": (pd.Timestamp("2025-02-28"), pd.Timestamp("2025-03-29")), "Natal": (pd.Timestamp("2025-12-15"), pd.Timestamp("2025-12-25"))}
# upcoming_events = {event: dates for event, dates in events_calendar.items() if dates[0] < HARI_INI + pd.Timedelta(days=90)}
# print(f"   - Konteks waktu disiapkan. Event mendatang: {list(upcoming_events.keys())}")


# # ==============================================================================
# # LANGKAH 2: DEFINISIKAN & JALANKAN MESIN ATURAN UNTUK BESARAN DISKON
# # ==============================================================================
# print("\n--- LANGKAH 2: Menjalankan Mesin Aturan untuk Menghitung Besaran Diskon ---")
# def round_discount(diskon):
#     if diskon is None or diskon < 0.05: return 0.05
#     return round((diskon * 100) / 5) * 5 / 100

# def get_recommendation_magnitude(row, upcoming_events):
#     strategi = row['rekomendasi_utama']
#     kategori, kedaluwarsa = row['kategori_produk'], row['hari_menuju_kedaluwarsa']
#     harga_kita, harga_kompetitor = row['harga_jual'], row['harga_kompetitor']
#     hari_jual_min = row['hari_jual_minimal']
#     if strategi == 'BOGO': return 0.50
#     if strategi == 'Tanpa Diskon': return 0.0
#     if 'Expired' in strategi:
#         if harga_kita > 0 and pd.notna(harga_kompetitor) and harga_kompetitor > 0:
#             harga_target = harga_kompetitor * 0.95
#             calc_besaran = 1 - (harga_target / harga_kita)
#             return round_discount(max(0.05, calc_besaran))
#         else: return 0.25
#     if 'Event' in strategi:
#         if harga_kita > 0 and pd.notna(harga_kompetitor) and harga_kompetitor > 0:
#             harga_target = harga_kompetitor * 0.98
#             calc_besaran = 1 - (harga_target / harga_kita)
#             return round_discount(max(0.05, calc_besaran))
#         else: return 0.15
#     if hari_jual_min <= 7: return 0.15
#     elif hari_jual_min <= 30: return 0.10
#     else: return 0.05
# df_final['rekomendasi_besaran'] = df_final.apply(lambda row: get_recommendation_magnitude(row, upcoming_events), axis=1)


# # ==============================================================================
# # LANGKAH 3: SEMPURNAKAN LABEL REKOMENDASI EVENT (DENGAN KAMUS YANG DISELARASKAN)
# # ==============================================================================
# print("\n--- LANGKAH 3: Menyempurnakan Detail Rekomendasi Event ---")
# df_final['rekomendasi_detail'] = df_final['rekomendasi_utama']

# # [DIUBAH] Kamus event diselaraskan dengan wawasan dari AI, 'Roti' ditambahkan ke Ramadan
# event_categories_map = {
#     'Ramadan': ['Sirup', 'Biskuit', 'Susu', 'Soda', 'Roti'],
#     'Natal': ['Cokelat', 'Biskuit', 'Soda', 'Sirup'],
#     'Tahun Baru': ['Soda', 'Cokelat', 'Sirup']
# }

# for event_name in upcoming_events:
#     relevant_categories = event_categories_map.get(event_name, [])
#     mask = (df_final['rekomendasi_utama'] == 'Event Based Discount') & (df_final['kategori_produk'].isin(relevant_categories))
#     df_final.loc[mask, 'rekomendasi_detail'] = f'Event Based ({event_name})'
# print("   - Detail rekomendasi event telah disempurnakan.")


# # ==============================================================================
# # LANGKAH 4: TAMPILKAN HASIL AKHIR GABUNGAN
# # ==============================================================================
# print("\n====================================================================")
# print("         HASIL AKHIR GABUNGAN (FINAL & SELARAS)           ")
# print("====================================================================")
# kolom_tampilan = ['id_produk', 'nama_produk', 'kategori_produk', 'rekomendasi_detail', 'rekomendasi_besaran', 'rata_rata_uplift_profit']
# display(df_final[kolom_tampilan].head(30).sort_values(by='rata_rata_uplift_profit', ascending=False).style.format({
#     'rekomendasi_besaran': '{:.1%}',
#     'rata_rata_uplift_profit': 'Rp {:,.0f}'
# }).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ==============================================================================
# LANGKAH 0 (BARU): DEFINISI FUNGSI & ANALISIS KATEGORI EVENT SECARA DINAMIS
# ==============================================================================
print("--- LANGKAH 0: Menganalisis Kategori Event Secara Dinamis dari Data Historis ---")

# Definisikan Kalender Event dengan tipe data yang benar (Timestamp)
events_calendar_for_analysis = {
    "Ramadan_2023": (pd.Timestamp("2023-03-22"), pd.Timestamp("2023-04-21")),
    "Natal_2023": (pd.Timestamp("2023-12-15"), pd.Timestamp("2023-12-25")),
    "Tahun Baru_2023": (pd.Timestamp("2023-12-26"), pd.Timestamp("2024-01-02")),
    "Ramadan_2024": (pd.Timestamp("2024-03-10"), pd.Timestamp("2024-04-09")),
    "Natal_2024": (pd.Timestamp("2024-12-15"), pd.Timestamp("2024-12-25")),
    "Tahun Baru_2024": (pd.Timestamp("2024-12-26"), pd.Timestamp("2025-01-02")),
}

# Definisikan Fungsi get_current_event yang akan kita gunakan
def get_current_event(date):
    for event, (start, end) in events_calendar_for_analysis.items():
        if start <= date <= end:
            return event.split('_')[0]
    if date.weekday() >= 4:
        return "Promo Akhir Pekan"
    return "Hari Biasa"

# Lakukan analisis (seperti kode sebelumnya)
df_analisis = df_transaksi.copy()
df_analisis['tanggal_transaksi'] = pd.to_datetime(df_analisis['tanggal_transaksi'])
df_analisis = pd.merge(df_analisis, df_produk[['id_produk', 'kategori_produk']], on='id_produk', how='left')
daily_sales_per_cat = df_analisis.groupby(['tanggal_transaksi', 'kategori_produk']).size().reset_index(name='penjualan')
daily_sales_per_cat['event'] = daily_sales_per_cat['tanggal_transaksi'].apply(get_current_event)
avg_sales_normal = daily_sales_per_cat[daily_sales_per_cat['event'].isin(['Hari Biasa', 'Promo Akhir Pekan'])]
avg_normal = avg_sales_normal.groupby('kategori_produk')['penjualan'].mean().to_dict()
event_categories_map = {}
list_event_utama = ['Ramadan', 'Natal', 'Tahun Baru']
LIFT_THRESHOLD = 1.2
for event in list_event_utama:
    avg_sales_event = daily_sales_per_cat[daily_sales_per_cat['event'] == event]
    if not avg_sales_event.empty:
        avg_event = avg_sales_event.groupby('kategori_produk')['penjualan'].mean()
        event_cats = [cat for cat, avg_sale in avg_event.items() if avg_sale > avg_normal.get(cat, 0) * LIFT_THRESHOLD]
        event_categories_map[event] = event_cats

print("   - Kamus Kategori Event Dinamis berhasil dibuat:")
print(event_categories_map)


# ==============================================================================
# LANGKAH 1: PERSIAPAN DATA & KONTEKS WAKTU (Tidak Berubah)
# ==============================================================================
print("\n--- MEMULAI PROSES PENGAYAAN REKOMENDASI (FINAL & SELARAS) ---")
df_final = df_agg_recommendation.copy()
print(f"   - Memulai dengan {len(df_final)} produk dari hasil T-Learner.")
df_final['id_produk'] = df_final['id_produk'].astype(str)
df_produk['id_produk'] = df_produk['id_produk'].astype(str)
fitur_tambahan = ['id_produk', 'harga_jual', 'harga_kompetitor', 'produk_musiman', 'hari_jual_minimal', 'expire_date']
df_final = pd.merge(df_final, df_produk[fitur_tambahan], on='id_produk', how='left')
HARI_INI = pd.to_datetime(df_transaksi['tanggal_transaksi']).max() + pd.Timedelta(days=1)
df_final['expire_date'] = pd.to_datetime(df_final['expire_date'])
df_final['hari_menuju_kedaluwarsa'] = (df_final['expire_date'] - HARI_INI).dt.days
# Gunakan kalender yang sama seperti di atas untuk konsistensi
upcoming_events_calendar = {
    "Ramadan": (pd.Timestamp("2025-02-28"), pd.Timestamp("2025-03-29")),
    "Natal": (pd.Timestamp("2025-12-15"), pd.Timestamp("2025-12-25")),
}
upcoming_events = {event: dates for event, dates in upcoming_events_calendar.items() if dates[0] < HARI_INI + pd.Timedelta(days=90)}
print(f"   - Konteks waktu disiapkan. Event mendatang: {list(upcoming_events.keys())}")


# ==============================================================================
# LANGKAH 2 & 3: DEFINISIKAN MESIN ATURAN DAN SEMPURNAKAN LABEL
# ==============================================================================
print("\n--- LANGKAH 2 & 3: Menjalankan Mesin Aturan dan Menyempurnakan Label ---")

def get_recommendation_magnitude(row, upcoming_events):
    # ... (fungsi ini tidak diubah) ...
    strategi = row['rekomendasi_utama']
    kategori, kedaluwarsa = row['kategori_produk'], row['hari_menuju_kedaluwarsa']
    harga_kita, harga_kompetitor = row['harga_jual'], row['harga_kompetitor']
    hari_jual_min = row['hari_jual_minimal']
    if strategi == 'BOGO': return 0.50
    if strategi == 'Tanpa Diskon': return 0.0
    if 'Expired' in strategi:
        if harga_kita > 0 and pd.notna(harga_kompetitor) and harga_kompetitor > 0:
            return round_discount(max(0.05, 1 - ((harga_kompetitor * 0.95) / harga_kita)))
        else: return 0.25
    if 'Event' in strategi:
        if harga_kita > 0 and pd.notna(harga_kompetitor) and harga_kompetitor > 0:
            return round_discount(max(0.05, 1 - ((harga_kompetitor * 0.98) / harga_kita)))
        else: return 0.15
    if hari_jual_min <= 7: return 0.15
    elif hari_jual_min <= 30: return 0.10
    else: return 0.05

def round_discount(diskon):
    if diskon is None or diskon < 0.05: return 0.05
    return round((diskon * 100) / 5) * 5 / 100

df_final['rekomendasi_besaran'] = df_final.apply(lambda row: get_recommendation_magnitude(row, upcoming_events), axis=1)
df_final['rekomendasi_detail'] = df_final['rekomendasi_utama']

for event_name in upcoming_events:
    relevant_categories = event_categories_map.get(event_name, [])
    mask = (df_final['rekomendasi_utama'] == 'Event Based Discount') & (df_final['kategori_produk'].isin(relevant_categories))
    df_final.loc[mask, 'rekomendasi_detail'] = f'Event Based ({event_name})'
print("   - Detail rekomendasi dan besaran berhasil dibuat.")

# ==============================================================================
# LANGKAH 4: TAMPILKAN HASIL AKHIR
# ==============================================================================
print("\n====================================================================")
print("         HASIL AKHIR GABUNGAN (FINAL & DINAMIS)           ")
print("====================================================================")
kolom_tampilan = ['id_produk', 'nama_produk', 'kategori_produk', 'rekomendasi_detail', 'rekomendasi_besaran', 'rata_rata_uplift_profit']
# display(df_final[kolom_tampilan].head(30).sort_values(by='rata_rata_uplift_profit', ascending=False).style.format({
#     'rekomendasi_besaran': '{:.1%}',
#     'rata_rata_uplift_profit': 'Rp {:,.0f}'
# }).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))

--- LANGKAH 0: Menganalisis Kategori Event Secara Dinamis dari Data Historis ---
   - Kamus Kategori Event Dinamis berhasil dibuat:
{'Ramadan': ['Biskuit', 'Buah Kering', 'Daging Segar', 'Kecap', 'Minyak Goreng', 'Sirup', 'Susu Kemasan', 'Teh'], 'Natal': ['Biskuit', 'Cokelat', 'Daging Segar', 'Mentega', 'Pasta', 'Sirup', 'Soda'], 'Tahun Baru': ['Cokelat', 'Daging Segar', 'Kacang', 'Kentang Goreng', 'Keripik', 'Nugget', 'Soda']}

--- MEMULAI PROSES PENGAYAAN REKOMENDASI (FINAL & SELARAS) ---
   - Memulai dengan 1000 produk dari hasil T-Learner.
   - Konteks waktu disiapkan. Event mendatang: ['Ramadan']

--- LANGKAH 2 & 3: Menjalankan Mesin Aturan dan Menyempurnakan Label ---
   - Detail rekomendasi dan besaran berhasil dibuat.

         HASIL AKHIR GABUNGAN (FINAL & DINAMIS)           


In [16]:
import pandas as pd
import numpy as np

print("--- MEMULAI PROSES FINALISASI REKOMENDASI DENGAN LOGIKA FALLBACK (VERSI DISEMPURNAKAN) ---")

# ==============================================================================
# LANGKAH 1: IDENTIFIKASI BARIS YANG PERLU DIPERBAIKI
# ==============================================================================
mask_perlu_diperbaiki = df_final['rekomendasi_detail'] == 'Event Based Discount'
jumlah_diperbaiki = mask_perlu_diperbaiki.sum()
print(f"   - Ditemukan {jumlah_diperbaiki} produk dengan rekomendasi 'Event Based' generik untuk dievaluasi ulang.")


# ==============================================================================
# LANGKAH 2: DEFINISIKAN & JALANKAN LOGIKA FALLBACK DENGAN ATURAN BOGO YANG LEBIH BAIK
# ==============================================================================
if jumlah_diperbaiki > 0:
    def get_fallback_recommendation(row):
        """
        Fungsi ini hanya dijalankan untuk produk yang rekomendasinya 'Event Based' tapi tidak cocok event.
        Ia akan menentukan tipe dan besaran diskon baru yang paling sesuai.
        """
        # [DIUBAH] Daftar kategori BOGO diperluas berdasarkan analisis.
        kategori_bogo = [
            'Biskuit', 'Soda', 'Cokelat', 'Sereal', 'Mie Instan', 'Jus Kemasan',
            'Teh', 'Pasta', 'Permen', 'Makaroni', 'Kuaci', 'Yogurt', 'Nugget',
            'Air Mineral', 'Minuman Isotonik', 'Keripik', 'Susu Kemasan',
            'Kopi Kemasan', 'Kacang', 'Ice Cream', 'Kentang Goreng', 'Sarden Kaleng', 'Kornet'
        ]

        # Prioritas 1: Apakah produk akan kedaluwarsa?
        if row['hari_menuju_kedaluwarsa'] <= 45:
            tipe_baru = "Expired Discount"
            if row['harga_jual'] > 0 and pd.notna(row['harga_kompetitor']) and row['harga_kompetitor'] > 0:
                besaran_baru = round_discount(max(0.05, 1 - ((row['harga_kompetitor'] * 0.95) / row['harga_jual'])))
            else:
                besaran_baru = 0.25 # Default
            return {'tipe_baru': tipe_baru, 'besaran_baru': besaran_baru}

        # Prioritas 2: Apakah cocok untuk BOGO?
        if row['kategori_produk'] in kategori_bogo:
            tipe_baru = "BOGO"
            besaran_baru = 0.50
            return {'tipe_baru': tipe_baru, 'besaran_baru': besaran_baru}

        # Prioritas 3 (Fallback Terakhir): Jadikan Generic Product Discount
        tipe_baru = "Generic Product Discount"
        if row['hari_jual_minimal'] <= 7:
            besaran_baru = 0.15
        else:
            besaran_baru = 0.10
        return {'tipe_baru': tipe_baru, 'besaran_baru': besaran_baru}

    print("   - Menerapkan logika fallback pada produk yang ditargetkan...")
    koreksi = df_final[mask_perlu_diperbaiki].apply(get_fallback_recommendation, axis=1)

    df_final.loc[mask_perlu_diperbaiki, 'rekomendasi_detail'] = [r['tipe_baru'] for r in koreksi]
    df_final.loc[mask_perlu_diperbaiki, 'rekomendasi_besaran'] = [r['besaran_baru'] for r in koreksi]

    print("   - Rekomendasi berhasil diperbarui dengan logika fallback yang disempurnakan.")
else:
    print("   - Tidak ada produk yang perlu dievaluasi ulang, semua sudah spesifik.")


# ==============================================================================
# LANGKAH 3: TAMPILKAN HASIL AKHIR YANG TELAH DISEMPURNAKAN
# ==============================================================================
print("\n====================================================================")
print("         HASIL AKHIR SETELAH PROSES FALLBACK (DISEMPURNAKAN)           ")
print("====================================================================")
kolom_tampilan = ['id_produk', 'nama_produk', 'kategori_produk', 'rekomendasi_detail', 'rekomendasi_besaran', 'rata_rata_uplift_profit']

display(df_final[kolom_tampilan].head(30).sort_values(by='rata_rata_uplift_profit', ascending=False).style.format({
    'rekomendasi_besaran': '{:.1%}',
    'rata_rata_uplift_profit': 'Rp {:,.0f}'
}).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))

--- MEMULAI PROSES FINALISASI REKOMENDASI DENGAN LOGIKA FALLBACK (VERSI DISEMPURNAKAN) ---
   - Ditemukan 236 produk dengan rekomendasi 'Event Based' generik untuk dievaluasi ulang.
   - Menerapkan logika fallback pada produk yang ditargetkan...
   - Rekomendasi berhasil diperbarui dengan logika fallback yang disempurnakan.

         HASIL AKHIR SETELAH PROSES FALLBACK (DISEMPURNAKAN)           


,id_produk,nama_produk,kategori_produk,rekomendasi_detail,rekomendasi_besaran,rata_rata_uplift_profit
0,P00087170,AbadiCV Daging Segar Spesial 50g,Daging Segar,Event Based (Ramadan),5.0%,"Rp 1,311"
1,P00084404,JayaPangan Beras Varian A 500g,Beras,Generic Product Discount,10.0%,Rp 825
2,P00054344,PTJaya Daging Segar Spesial 1000g,Daging Segar,Event Based (Ramadan),5.0%,Rp 761
3,P00055495,AbadiAbadi Seafood Segar Varian B 750g,Seafood Segar,Generic Product Discount,15.0%,Rp 730
4,P00083371,AgroPT Seafood Segar Varian C 200g,Seafood Segar,Generic Product Discount,15.0%,Rp 603
5,P00074514,NusantaraPT Seafood Segar Ekstra 200g,Seafood Segar,Generic Product Discount,15.0%,Rp 595
6,P00065969,JayaPangan Seafood Segar Varian C 50g,Seafood Segar,Generic Product Discount,15.0%,Rp 542
7,P00071462,PanganIndo Buah Kering Spesial 100g,Buah Kering,Event Based (Ramadan),15.0%,Rp 537
8,P00065063,SentosaIndo Buah Kering Varian B 100g,Buah Kering,Event Based (Ramadan),15.0%,Rp 457
9,P00079809,AgroCV Daging Segar Ekstra 750g,Daging Segar,Event Based (Ramadan),5.0%,Rp 450


In [17]:
df_rekomendasi_final = (
    df_final[kolom_tampilan]
    .sort_values(by='rata_rata_uplift_profit', ascending=False)
    .head(30)
)
df_rekomendasi_final['rata_rata_uplift_profit'] = df_rekomendasi_final['rata_rata_uplift_profit'].round(0).astype(int)

In [18]:
df_rekomendasi_final

,id_produk,nama_produk,kategori_produk,rekomendasi_detail,rekomendasi_besaran,rata_rata_uplift_profit
0,P00087170,AbadiCV Daging Segar Spesial 50g,Daging Segar,Event Based (Ramadan),0.05,1311
1,P00084404,JayaPangan Beras Varian A 500g,Beras,Generic Product Discount,0.10,825
2,P00054344,PTJaya Daging Segar Spesial 1000g,Daging Segar,Event Based (Ramadan),0.05,761
3,P00055495,AbadiAbadi Seafood Segar Varian B 750g,Seafood Segar,Generic Product Discount,0.15,730
4,P00083371,AgroPT Seafood Segar Varian C 200g,Seafood Segar,Generic Product Discount,0.15,603
5,P00074514,NusantaraPT Seafood Segar Ekstra 200g,Seafood Segar,Generic Product Discount,0.15,595
6,P00065969,JayaPangan Seafood Segar Varian C 50g,Seafood Segar,Generic Product Discount,0.15,542
7,P00071462,PanganIndo Buah Kering Spesial 100g,Buah Kering,Event Based (Ramadan),0.15,537
8,P00065063,SentosaIndo Buah Kering Varian B 100g,Buah Kering,Event Based (Ramadan),0.15,457
9,P00079809,AgroCV Daging Segar Ekstra 750g,Daging Segar,Event Based (Ramadan),0.05,450


## Model Penentu Waktu

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ==============================================================================
# LANGKAH 1: INSTALASI, IMPOR, DAN PEMISAHAN STRATEGI
# ==============================================================================
print("--- LANGKAH 1: Instalasi, Impor & Pemisahan Strategi ---")

# Instalasi library yang dibutuhkan
!pip install -q pytorch-forecasting

import torch
import pytorch_lightning as pl
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer

# Pastikan tipe data kunci konsisten
df_rekomendasi_final['id_produk'] = df_rekomendasi_final['id_produk'].astype(str)
df_transaksi['id_produk'] = df_transaksi['id_produk'].astype(str)

# Dapatkan daftar ID produk yang memiliki sejarah transaksi
produk_dengan_histori_ids = df_transaksi['id_produk'].unique()

# Pisahkan daftar rekomendasi menjadi dua grup
df_rekomendasi_dengan_histori = df_rekomendasi_final[df_rekomendasi_final['id_produk'].isin(produk_dengan_histori_ids)].copy()
df_rekomendasi_cold_start = df_rekomendasi_final[~df_rekomendasi_final['id_produk'].isin(produk_dengan_histori_ids)].copy()

print(f"\n   - Ditemukan {len(df_rekomendasi_dengan_histori)} produk dengan histori (akan menggunakan TFT).")
print(f"   - Ditemukan {len(df_rekomendasi_cold_start)} produk 'Cold Start' (akan menggunakan Mesin Aturan).")


# ==============================================================================
# LANGKAH 2: PROSES CABANG A - PERAMALAN TFT UNTUK PRODUK DENGAN HISTORI
# ==============================================================================
print("\n--- LANGKAH 2: Memproses Produk dengan Histori menggunakan TFT ---")
kalender_promosi_tft = []

# Hanya jalankan TFT jika ada produk dengan histori
if not df_rekomendasi_dengan_histori.empty:
    # 2a. Persiapan Data Deret Waktu
    list_produk_target_tft = df_rekomendasi_dengan_histori['id_produk'].unique().tolist()
    df_transaksi_filtered = df_transaksi[df_transaksi['id_produk'].isin(list_produk_target_tft)].copy()
    df_transaksi_filtered['tanggal_transaksi'] = pd.to_datetime(df_transaksi_filtered['tanggal_transaksi'])
    df_daily_sales = df_transaksi_filtered.groupby(['tanggal_transaksi', 'id_produk']).size().reset_index(name='penjualan_harian')
    start_date, end_date = df_transaksi_filtered['tanggal_transaksi'].min(), df_transaksi_filtered['tanggal_transaksi'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    multi_index = pd.MultiIndex.from_product([all_dates, list_produk_target_tft], names=['tanggal_transaksi', 'id_produk'])
    df_timeseries = pd.DataFrame(index=multi_index).reset_index()
    df_timeseries = pd.merge(df_timeseries, df_daily_sales, on=['tanggal_transaksi', 'id_produk'], how='left').fillna(0)
    df_timeseries = pd.merge(df_timeseries, df_produk, on='id_produk', how='left')
    df_timeseries['time_idx'] = (df_timeseries['tanggal_transaksi'] - df_timeseries['tanggal_transaksi'].min()).dt.days
    df_timeseries['bulan'] = df_timeseries['tanggal_transaksi'].dt.month
    df_timeseries['hari_dalam_minggu'] = df_timeseries['tanggal_transaksi'].dt.dayofweek

    # 2b. Membuat TimeSeriesDataSet & Melatih Model
    max_prediction_length, max_encoder_length = 90, 180
    training_cutoff = df_timeseries["time_idx"].max() - max_prediction_length
    training_dataset = TimeSeriesDataSet(
        df_timeseries[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx", target="penjualan_harian", group_ids=["id_produk"],
        max_encoder_length=max_encoder_length, max_prediction_length=max_prediction_length,
        static_categoricals=["kategori_produk"], static_reals=["harga_jual"],
        time_varying_known_reals=["bulan", "hari_dalam_minggu"], time_varying_unknown_reals=["penjualan_harian"],
        target_normalizer=GroupNormalizer(groups=["id_produk"]),
    )
    train_dataloader = training_dataset.to_dataloader(train=True, batch_size=16, num_workers=0)
    trainer = pl.Trainer(accelerator="cpu", max_epochs=10, default_root_dir="logs/", enable_progress_bar=False, enable_model_summary=False)
    tft = TemporalFusionTransformer.from_dataset(training_dataset, learning_rate=0.03, hidden_size=8, attention_head_size=1, dropout=0.1, hidden_continuous_size=4)
    trainer.fit(tft, train_dataloaders=train_dataloader)
    print("   - Model TFT berhasil dilatih.")

    # 2c. Prediksi Masa Depan (Lengkap & Asli)
    print("   - Melakukan peramalan 90 hari ke depan...")
    predictions = tft.predict(training_dataset, mode="prediction", return_index=True)
    df_predictions_flat = predictions.output.squeeze().reset_index()
    df_predictions_flat['tanggal_prediksi'] = df_predictions_flat['time_idx'].apply(lambda x: end_date + timedelta(days=x - df_timeseries["time_idx"].max()))

    # 2d. Penjadwalan Cerdas Berbasis Prediksi
    print("   - Menerapkan logika penjadwalan cerdas...")
    HARI_INI = end_date.date() + timedelta(days=1)
    for _, row in df_rekomendasi_dengan_histori.iterrows():
        id_p, rekomendasi = row['id_produk'], row['rekomendasi_detail']
        prediksi_produk = df_predictions_flat[df_predictions_flat['id_produk'] == id_p]
        start_date, end_date_promo = HARI_INI, HARI_INI + timedelta(days=6) # Default

        if "Event Based (Ramadan)" in rekomendasi:
            tgl_ramadan = datetime.strptime("2025-02-28", "%Y-%m-%d")
            # Cari minggu dengan permintaan TERENDAH sebelum Ramadan
            pra_ramadan = prediksi_produk[prediksi_produk['tanggal_prediksi'] < tgl_ramadan]
            if not pra_ramadan.empty:
                minggu_terendah_start = pra_ramadan.rolling(7).mean().idxmin()[0]
                start_date = prediksi_produk.loc[minggu_terendah_start, 'tanggal_prediksi'].date()
                end_date_promo = start_date + timedelta(days=13) # Durasi 2 minggu
        elif "BOGO" in rekomendasi:
            # Cari akhir pekan dengan permintaan terendah
            akhir_pekan = prediksi_produk[prediksi_produk['tanggal_prediksi'].dt.dayofweek.isin([4,5,6])]
            if not akhir_pekan.empty:
                jumat_terbaik = akhir_pekan.sort_values(by=0).iloc[0]['tanggal_prediksi']
                start_date = jumat_terbaik.date()
                end_date_promo = start_date + timedelta(days=2) # Durasi 3 hari

        kalender_promosi_tft.append({'id_produk': id_p, 'start_date': start_date, 'end_date': end_date_promo})
else:
    print("   - Tidak ada produk dengan histori untuk diproses dengan TFT.")


# ==============================================================================
# LANGKAH 3: PROSES CABANG B - MESIN ATURAN UNTUK PRODUK "COLD START"
# ==============================================================================
print("\n--- LANGKAH 3: Memproses Produk 'Cold Start' menggunakan Mesin Aturan ---")
kalender_promosi_aturan = []
HARI_INI = pd.to_datetime(df_transaksi['tanggal_transaksi']).max().date() + timedelta(days=1)

for _, row in df_rekomendasi_cold_start.iterrows():
    rekomendasi = row['rekomendasi_detail']
    start_date, end_date_promo = None, None
    if "Event Based (Ramadan)" in rekomendasi:
        start_date = datetime.strptime("2025-02-21", "%Y-%m-%d").date() # Mulai 1 minggu sebelum
        end_date_promo = start_date + timedelta(days=13)
    elif "Expired" in rekomendasi:
        bulan_depan = (HARI_INI + timedelta(days=30)).replace(day=1)
        jumat_pertama = bulan_depan + timedelta(days=(4 - bulan_depan.weekday() + 7) % 7)
        start_date, end_date_promo = jumat_pertama, jumat_pertama + timedelta(days=2)
    else: # BOGO atau Generic
        bulan_depan = (HARI_INI + timedelta(days=30)).replace(day=1)
        jumat_pertama = bulan_depan + timedelta(days=(4 - bulan_depan.weekday() + 7) % 7)
        start_date, end_date_promo = jumat_pertama + timedelta(days=7), jumat_pertama + timedelta(days=9)
    kalender_promosi_aturan.append({'id_produk': row['id_produk'], 'start_date': start_date, 'end_date': end_date_promo})


# ==============================================================================
# LANGKAH 4: GABUNGKAN HASIL DAN TAMPILKAN KALENDER FINAL
# ==============================================================================
print("\n--- LANGKAH 4: Menggabungkan & Menampilkan Hasil ---")

df_kalender_tft = pd.DataFrame(kalender_promosi_tft)
df_kalender_aturan = pd.DataFrame(kalender_promosi_aturan)
df_kalender_final = pd.concat([df_kalender_tft, df_kalender_aturan], ignore_index=True)
df_kalender_final = pd.merge(df_kalender_final, df_rekomendasi_final, on='id_produk', how='left')

print("\n====================================================================")
print("             HASIL AKHIR MODEL 3: KALENDER PROMOSI              ")
print("====================================================================")
kolom_tampilan = ['id_produk', 'nama_produk', 'rekomendasi_detail', 'rekomendasi_besaran', 'start_date', 'end_date']
display(df_kalender_final[kolom_tampilan].sort_values(by='start_date'))

--- LANGKAH 1: Instalasi, Impor & Pemisahan Strategi ---

   - Ditemukan 0 produk dengan histori (akan menggunakan TFT).
   - Ditemukan 30 produk 'Cold Start' (akan menggunakan Mesin Aturan).

--- LANGKAH 2: Memproses Produk dengan Histori menggunakan TFT ---
   - Tidak ada produk dengan histori untuk diproses dengan TFT.

--- LANGKAH 3: Memproses Produk 'Cold Start' menggunakan Mesin Aturan ---

--- LANGKAH 4: Menggabungkan & Menampilkan Hasil ---

             HASIL AKHIR MODEL 3: KALENDER PROMOSI              


,id_produk,nama_produk,rekomendasi_detail,rekomendasi_besaran,start_date,end_date
0,P00087170,AbadiCV Daging Segar Spesial 50g,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
23,P00087172,IndoAgro Daging Segar Varian C 200g,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
22,P00071404,JayaNusantara Buah Kering Varian C 100g,Event Based (Ramadan),0.10,2025-02-21,2025-03-06
21,P00081374,MakmurJaya Susu Kemasan Varian A 150ml,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
19,P00076982,CVMakmur Teh Spesial 250g,Event Based (Ramadan),0.10,2025-02-21,2025-03-06
18,P00070716,JayaPT Daging Segar Ekstra 200g,Event Based (Ramadan),0.15,2025-02-21,2025-03-06
17,P00079196,CVCV Susu Kemasan Spesial 1000ml,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
16,P00073767,AgroCV Susu Kemasan Spesial 150ml,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
15,P00075126,IndoNusantara Minyak Goreng Varian A 1000g,Event Based (Ramadan),0.05,2025-02-21,2025-03-06
13,P00054288,IndoAgro Daging Segar Ekstra 100g,Event Based (Ramadan),0.10,2025-02-21,2025-03-06


# OUTPUT FINAL

In [20]:
import pandas as pd

# ==============================================================================
# LANGKAH FINAL (VERSI PERBAIKAN): MEMBUAT OUTPUT RENCANA PROMOSI
# ==============================================================================
print("--- Menggunakan df_kalender_final yang sudah lengkap dari Model 3 ---")

# Asumsikan df_kalender_final sudah ada dari eksekusi Model 3 sebelumnya

# 1. Definisikan kolom final yang kita inginkan
# Nama-nama ini sudah ada di dalam df_kalender_final
kolom_final = [
    'id_produk',
    'nama_produk',
    'kategori_produk',
    'rekomendasi_detail',
    'rekomendasi_besaran',
    'start_date',
    'end_date',
    'rata_rata_uplift_profit'
]

# 2. Buat DataFrame baru hanya dengan memilih dan menyusun ulang kolom tersebut
df_rencana_promosi = df_kalender_final[kolom_final].copy()

# 3. (Opsional tapi direkomendasikan) Simpan ke file CSV
try:
    df_rencana_promosi.to_csv('rencana_promosi_final.csv', index=False)
    print("\n   - DataFrame final berhasil disimpan ke 'rencana_promosi_final.csv'")
except Exception as e:
    print(f"   - Gagal menyimpan ke CSV. Error: {e}")

# 4. Tampilkan hasil akhir
print("\n====================================================================")
print("         RENCANA PROMOSI FINAL (SIAP UNTUK API)           ")
print("====================================================================")
display(df_rencana_promosi.sort_values(by='start_date').style.format({
    'rekomendasi_besaran': '{:.1%}',
    'rata_rata_uplift_profit': 'Rp {:,.0f}'
}).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))

--- Menggunakan df_kalender_final yang sudah lengkap dari Model 3 ---

   - DataFrame final berhasil disimpan ke 'rencana_promosi_final.csv'

         RENCANA PROMOSI FINAL (SIAP UNTUK API)           


,id_produk,nama_produk,kategori_produk,rekomendasi_detail,rekomendasi_besaran,start_date,end_date,rata_rata_uplift_profit
0,P00087170,AbadiCV Daging Segar Spesial 50g,Daging Segar,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,"Rp 1,311"
23,P00087172,IndoAgro Daging Segar Varian C 200g,Daging Segar,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,Rp 221
22,P00071404,JayaNusantara Buah Kering Varian C 100g,Buah Kering,Event Based (Ramadan),10.0%,2025-02-21,2025-03-06,Rp 230
21,P00081374,MakmurJaya Susu Kemasan Varian A 150ml,Susu Kemasan,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,Rp 238
19,P00076982,CVMakmur Teh Spesial 250g,Teh,Event Based (Ramadan),10.0%,2025-02-21,2025-03-06,Rp 256
18,P00070716,JayaPT Daging Segar Ekstra 200g,Daging Segar,Event Based (Ramadan),15.0%,2025-02-21,2025-03-06,Rp 259
17,P00079196,CVCV Susu Kemasan Spesial 1000ml,Susu Kemasan,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,Rp 274
16,P00073767,AgroCV Susu Kemasan Spesial 150ml,Susu Kemasan,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,Rp 282
15,P00075126,IndoNusantara Minyak Goreng Varian A 1000g,Minyak Goreng,Event Based (Ramadan),5.0%,2025-02-21,2025-03-06,Rp 282
13,P00054288,IndoAgro Daging Segar Ekstra 100g,Daging Segar,Event Based (Ramadan),10.0%,2025-02-21,2025-03-06,Rp 348
